In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

On lit le dataset:

In [ ]:
df = pd.read_csv("../input/bostonhoustingmlnd/housing.csv")
df.head(10).T

L'une des interrogations principales sur ce dataset concerne le sens donné aux différentes variables "LSTAT", "PTRATIO" et "MEDV".

Avec ces données, on ne peut pas placer les maisons sur une carte, ni voir l'évolution du prix par date.

**Recherche de corrélations**

In [ ]:
tabcorr = df.corr()  

In [ ]:
tabcorr

On va afficher ces corrélations sur une heatmap

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm")

On peut tracer uniquement le dendrogramme des corrélations entre les caractéristiques. On utilise une forme "condensée" de la matrice de corrélation. linkage permet de calculer les distances entre caractéristiques à partir des corrélations

In [ ]:
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)

On va désormais s'intéresser plus particulièrement au prix.

In [ ]:
correlations = tabcorr.MEDV
print(correlations)

On élimine la ligne MV elle-même (qui est forcément à 1) :

In [ ]:
correlations = correlations.drop(['MEDV'],axis=0)

Les corrélations fortement négatives sont aussi significatives que les positives ; on considère donc les valeurs absolues, et on trie par ordre décroissant :

In [ ]:
print(abs(correlations).sort_values(ascending=False))

On voit que la plus grosse corrélation concerne "LSTAT", mais que les 3 paramètres sont fortement corrélés au prix.

On s'intéresse désormais à la régression linéaire multiple.

In [ ]:
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train, y_train)            
y_pred = lm.predict(X_test) 

On trace le nuage de points pour comparer la prédiction et les résultats attendus :

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

On teste différentes méthodes pour visualiser notre erreur: la distribution de l'erreur avec seaborn ainsi que le score R2.

In [ ]:
sns.distplot(y_test-y_pred)

In [ ]:
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

**Régression par forêts aléatoires**

In [ ]:
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))
rf.score(X_test,y_test)

Enfin, on va utiliser la méthode XGBoost pour voir si elle est plus précise.

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")